In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # 選擇 kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c intro-ml-2025-nccu-task-5


  0% 0.00/3.25M [00:00<?, ?B/s]
100% 3.25M/3.25M [00:00<00:00, 430MB/s]


In [ ]:
!unzip intro-ml-2025-nccu-task-5.zip -d task5

Archive:  intro-ml-2025-nccu-task-5.zip
  inflating: task5/task5_sample_submissions.csv  
  inflating: task5/task5_test.csv    
  inflating: task5/task5_train.csv   


In [ ]:
import pandas as pd

train = pd.read_csv("task5/task5_train.csv")
test = pd.read_csv("task5/task5_test.csv")
sample = pd.read_csv("task5/task5_sample_submissions.csv")

train.head()


,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20,label
0,-1.926460,5.569192,4.037518,-3.060529,-3.700440,5.995797,-5.399668,-3.025864,3.952591,8.484403,...,0.399694,0.450825,3.568744,8.614852,2.602650,3.293376,-1.900946,1.815461,-8.403425,D
1,-3.588237,3.821008,3.068594,0.918297,-3.543245,7.690221,-4.251693,-3.599600,2.081101,6.961915,...,4.171805,-0.425526,2.666164,10.823750,0.904388,3.270731,-4.574082,1.133923,-9.752564,D
2,-4.021023,-2.355731,0.061592,1.280351,-2.608264,3.895480,0.963985,1.383272,4.209569,2.387123,...,-5.648358,1.211678,-1.253597,-0.189038,-0.822591,13.022118,-5.951954,-9.642198,0.263612,H
3,-0.033057,1.507021,-0.470896,-2.640740,-0.921980,3.672495,1.555421,3.293253,-1.585033,0.807233,...,2.805356,-0.337240,-2.787551,-0.066639,0.287358,-0.747471,0.523760,-1.549452,0.492427,F
4,2.767705,-0.312920,8.787348,-7.869882,-3.765386,0.545452,5.139767,11.517443,5.427437,2.360541,...,-2.783335,-2.548858,2.652028,0.384332,-10.819073,-2.850848,6.178687,5.504717,1.566702,I


In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
sample.head()

In [ ]:
!pip install xgboost -q

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

# ================================
# 1. 切特徵與標籤
# ================================
X = train.drop(columns=["label"])
y = train["label"]

# 將 A/B/C/D 轉成 0/1/2/3
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# ================================
# 2. 切 train / valid
# ================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y_encoded,
    test_size=0.1,
    random_state=42,
    shuffle=True
)

# ================================
# 3. 建立多分類模型
# ================================
model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multi:softprob",
    num_class=len(le.classes_),   # = 4
    n_jobs=-1,
    random_state=42
)

# ================================
# 4. 訓練
# ================================
model.fit(
    X_train,
    y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=False
)

# ================================
# 5. 預測與評估
# ================================
y_pred = model.predict(X_valid)

acc = accuracy_score(y_valid, y_pred)
print("Valid Accuracy:", acc)

# 如果你需要預測 label 字串
y_pred_label = le.inverse_transform(y_pred)



Valid Accuracy: 1.0


In [ ]:
# ================================
# 7. 對 test 做預測，產生 submission（多分類）
# ================================
# test 應該有：id, x_1 ~ x_10
X_test = test.drop(columns=["id"])

# predict -> 0/1/2/3
test_pred = model.predict(X_test)

# 把 0/1/2/3 轉回 A/B/C/D
test_pred_labels = le.inverse_transform(test_pred)

submission = pd.DataFrame({
    "id": test["id"],
    "label": test_pred_labels
})

print("Submission head:")
print(submission.head())

# 儲存
submission.to_csv("submission_xgb_baseline_task4.csv", index=False)
print("Saved to submission_xgb_baseline_task4.csv")

files.download("submission_xgb_baseline_task4.csv")

Submission head:
   id label
0   1     E
1   2     C
2   3     J
3   4     E
4   5     C
Saved to submission_xgb_baseline_task4.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>